# Αναγνώριση Οντοτήτων (NER)

Αυτό το σημειωματάριο προέρχεται από το [Πρόγραμμα Σπουδών AI για Αρχάριους](http://aka.ms/ai-beginners).

Σε αυτό το παράδειγμα, θα μάθουμε πώς να εκπαιδεύσουμε ένα μοντέλο NER χρησιμοποιώντας το Dataset [Annotated Corpus for Named Entity Recognition](https://www.kaggle.com/datasets/abhinavwalia95/entity-annotated-corpus) από το Kaggle. Πριν προχωρήσετε, παρακαλώ κατεβάστε το αρχείο [ner_dataset.csv](https://www.kaggle.com/datasets/abhinavwalia95/entity-annotated-corpus?resource=download&select=ner_dataset.csv) στον τρέχοντα κατάλογο.


In [62]:
import pandas as pd
from tensorflow import keras
import numpy as np

## Προετοιμασία του Συνόλου Δεδομένων

Θα ξεκινήσουμε διαβάζοντας το σύνολο δεδομένων σε ένα dataframe. Αν θέλετε να μάθετε περισσότερα για τη χρήση του Pandas, επισκεφθείτε ένα [μάθημα για την επεξεργασία δεδομένων](https://github.com/microsoft/Data-Science-For-Beginners/tree/main/2-Working-With-Data/07-python) στον οδηγό μας [Επιστήμη Δεδομένων για Αρχάριους](http://aka.ms/datascience-beginners)


In [3]:
df = pd.read_csv('ner_dataset.csv',encoding='unicode-escape')
df.head()

,Sentence #,Word,POS,Tag
0,Sentence: 1,Thousands,NNS,O
1,NaN,of,IN,O
2,NaN,demonstrators,NNS,O
3,NaN,have,VBP,O
4,NaN,marched,VBN,O


Ας πάρουμε μοναδικές ετικέτες και να δημιουργήσουμε λεξικά αναζήτησης που μπορούμε να χρησιμοποιήσουμε για να μετατρέψουμε τις ετικέτες σε αριθμούς κατηγοριών:


In [4]:
tags = df.Tag.unique()
tags

array(['O', 'B-geo', 'B-gpe', 'B-per', 'I-geo', 'B-org', 'I-org', 'B-tim',
       'B-art', 'I-art', 'I-per', 'I-gpe', 'I-tim', 'B-nat', 'B-eve',
       'I-eve', 'I-nat'], dtype=object)

In [8]:
id2tag = dict(enumerate(tags))
tag2id = { v : k for k,v in id2tag.items() }

id2tag[0]

'O'

Τώρα πρέπει να κάνουμε το ίδιο με το λεξιλόγιο. Για απλότητα, θα δημιουργήσουμε λεξιλόγιο χωρίς να λαμβάνουμε υπόψη τη συχνότητα των λέξεων· στην πραγματική ζωή μπορεί να θέλετε να χρησιμοποιήσετε τον vectorizer του Keras και να περιορίσετε τον αριθμό των λέξεων.


In [14]:
vocab = set(df['Word'].apply(lambda x: x.lower()))
id2word = { i+1 : v for i,v in enumerate(vocab) }
id2word[0] = '<UNK>'
vocab.add('<UNK>')
word2id = { v : k for k,v in id2word.items() }

Πρέπει να δημιουργήσουμε ένα σύνολο δεδομένων προτάσεων για εκπαίδευση. Ας επαναλάβουμε το αρχικό σύνολο δεδομένων και να διαχωρίσουμε όλες τις μεμονωμένες προτάσεις σε `X` (λίστες λέξεων) και `Y` (λίστα συμβόλων):


In [41]:
X,Y = [],[]
s,t = [],[]
for i,row in df[['Sentence #','Word','Tag']].iterrows():
    if pd.isna(row['Sentence #']):
        s.append(row['Word'])
        t.append(row['Tag'])
    else:
        if len(s)>0:
            X.append(s)
            Y.append(t)
        s,t = [row['Word']],[row['Tag']]
X.append(s)
Y.append(t)


In [93]:
def vectorize(seq):
    return [word2id[x.lower()] for x in seq]

def tagify(seq):
    return [tag2id[x] for x in seq]

Xv = list(map(vectorize,X))
Yv = list(map(tagify,Y))

Xv[0], Yv[0]

([10386,
  23515,
  4134,
  29620,
  7954,
  13583,
  21193,
  12222,
  27322,
  18258,
  5815,
  15880,
  5355,
  25242,
  31327,
  18258,
  27067,
  23515,
  26444,
  14412,
  358,
  26551,
  5011,
  30558],
 [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0])

Για απλότητα, θα συμπληρώσουμε όλες τις προτάσεις με 0 tokens μέχρι το μέγιστο μήκος. Στην πραγματική ζωή, μπορεί να θέλουμε να χρησιμοποιήσουμε μια πιο έξυπνη στρατηγική και να συμπληρώσουμε τις ακολουθίες μόνο μέσα σε ένα minibatch.


In [51]:
X_data = keras.preprocessing.sequence.pad_sequences(Xv,padding='post')
Y_data = keras.preprocessing.sequence.pad_sequences(Yv,padding='post')

## Ορισμός Δικτύου Κατηγοριοποίησης Συμβόλων

Θα χρησιμοποιήσουμε ένα δίκτυο bidirectional LSTM δύο επιπέδων για την κατηγοριοποίηση συμβόλων. Για να εφαρμόσουμε έναν πυκνό ταξινομητή σε κάθε έξοδο του τελευταίου επιπέδου LSTM, θα χρησιμοποιήσουμε την κατασκευή `TimeDistributed`, η οποία αναπαράγει το ίδιο πυκνό επίπεδο σε κάθε έξοδο του LSTM σε κάθε βήμα:


In [94]:
maxlen = X_data.shape[1]
vocab_size = len(vocab)
num_tags = len(tags)
model = keras.models.Sequential([
    keras.layers.Embedding(vocab_size, 300, input_length=maxlen),
    keras.layers.Bidirectional(keras.layers.LSTM(units=100, activation='tanh', return_sequences=True)),
    keras.layers.Bidirectional(keras.layers.LSTM(units=100, activation='tanh', return_sequences=True)),
    keras.layers.TimeDistributed(keras.layers.Dense(num_tags, activation='softmax'))
])
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['acc'])
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_4 (Embedding)     (None, 104, 300)          9545400   
                                                                 
 bidirectional_6 (Bidirectio  (None, 104, 200)         320800    
 nal)                                                            
                                                                 
 bidirectional_7 (Bidirectio  (None, 104, 200)         240800    
 nal)                                                            
                                                                 
 time_distributed_3 (TimeDis  (None, 104, 17)          3417      
 tributed)                                                       
                                                                 
Total params: 10,110,417
Trainable params: 10,110,417
Non-trainable params: 0
__________________________________________

Σημειώστε εδώ ότι καθορίζουμε ρητά το `maxlen` για το σύνολο δεδομένων μας - σε περίπτωση που θέλουμε το δίκτυο να μπορεί να χειριστεί ακολουθίες μεταβλητού μήκους, πρέπει να είμαστε λίγο πιο έξυπνοι κατά τον ορισμό του δικτύου.

Ας εκπαιδεύσουμε τώρα το μοντέλο. Για λόγους ταχύτητας, θα εκπαιδεύσουμε μόνο για μία εποχή, αλλά μπορείτε να δοκιμάσετε να εκπαιδεύσετε για μεγαλύτερο χρονικό διάστημα. Επίσης, ίσως θελήσετε να διαχωρίσετε ένα μέρος του συνόλου δεδομένων ως σύνολο εκπαίδευσης, για να παρατηρήσετε την ακρίβεια επικύρωσης.


In [57]:
model.fit(X_data,Y_data)

1499/1499 [==============================] - 740s 488ms/step - loss: 0.0667 - acc: 0.9841


## Δοκιμή του Αποτελέσματος

Ας δούμε τώρα πώς λειτουργεί το μοντέλο αναγνώρισης οντοτήτων σε μια δείγμα πρόταση:


In [91]:
sent = 'John Smith went to Paris to attend a conference in cancer development institute'
words = sent.lower().split()
v = keras.preprocessing.sequence.pad_sequences([[word2id[x] for x in words]],padding='post',maxlen=maxlen)
res = model(v)[0]

In [92]:
r = np.argmax(res.numpy(),axis=1)
for i,w in zip(r,words):
    print(f"{w} -> {id2tag[i]}")

john -> B-per
smith -> I-per
went -> O
to -> O
paris -> B-geo
to -> O
attend -> O
a -> O
conference -> O
in -> O
cancer -> B-org
development -> I-org
institute -> I-org


## Συμπέρασμα

Ακόμα και ένα απλό μοντέλο LSTM μπορεί να δείξει ικανοποιητικά αποτελέσματα στο NER. Ωστόσο, για να πετύχετε πολύ καλύτερα αποτελέσματα, ίσως θελήσετε να χρησιμοποιήσετε μεγάλα προεκπαιδευμένα γλωσσικά μοντέλα όπως το BERT. Η εκπαίδευση του BERT για NER χρησιμοποιώντας τη βιβλιοθήκη Huggingface Transformers περιγράφεται [εδώ](https://huggingface.co/course/chapter7/2?fw=pt).



---

**Αποποίηση Ευθύνης**:  
Αυτό το έγγραφο έχει μεταφραστεί χρησιμοποιώντας την υπηρεσία αυτόματης μετάφρασης AI [Co-op Translator](https://github.com/Azure/co-op-translator). Παρόλο που καταβάλλουμε προσπάθειες για ακρίβεια, παρακαλούμε να έχετε υπόψη ότι οι αυτόματες μεταφράσεις ενδέχεται να περιέχουν σφάλματα ή ανακρίβειες. Το πρωτότυπο έγγραφο στη μητρική του γλώσσα θα πρέπει να θεωρείται η αυθεντική πηγή. Για κρίσιμες πληροφορίες, συνιστάται επαγγελματική ανθρώπινη μετάφραση. Δεν φέρουμε ευθύνη για τυχόν παρεξηγήσεις ή εσφαλμένες ερμηνείες που προκύπτουν από τη χρήση αυτής της μετάφρασης.
